We first construct the simulated bibliographies so that they have the descriptions. 

In [ ]:
# simulation on bilography data
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def simulate_biographies(
    n=2000,
    k=6,
    tau=1.0,
    seed=7
):
    rng = np.random.default_rng(seed)
    U = rng.normal(size=(n, k))

    # Treatment depends on U (confounding) but not deterministic
    beta = rng.normal(size=k)
    p = sigmoid(U @ beta / 1.5)  # soften to keep overlap
    T = rng.binomial(1, p, size=n)

    # Outcome depends on T and U
    gamma = rng.normal(size=k)
    Y = tau * T + U @ gamma + rng.normal(scale=1.0, size=n)

    # Convert U into coarse “traits” that show up as text confounders
    # (topic-ish phrases)
    def trait_phrases(u_row):
        phrases = []
        if u_row[0] > 0: phrases.append("worked in finance and budgeting")
        else:            phrases.append("worked in education and mentoring")

        if u_row[1] > 0: phrases.append("focused on data and analytics")
        else:            phrases.append("focused on community outreach")

        if u_row[2] > 0: phrases.append("earned an MBA and led teams")
        else:            phrases.append("earned a BA and supported operations")

        if u_row[3] > 0: phrases.append("enjoys long-distance running")
        else:            phrases.append("enjoys painting and museums")

        if u_row[4] > 0: phrases.append("has international experience in Asia")
        else:            phrases.append("has international experience in Europe")

        if u_row[5] > 0: phrases.append("volunteers with youth programs")
        else:            phrases.append("volunteers with environmental groups")
        return phrases

    # Treatment feature: a specific textual attribute
    treat_phrase = "served in the military for four years"

    texts = []
    for i in range(n):
        base = trait_phrases(U[i])
        # Add some random fluff to prevent trivial keyword-only embeddings
        fluff = ["values teamwork", "is detail-oriented", "likes public speaking", "enjoys reading history"]
        rng.shuffle(fluff)
        parts = [
            "Biography:",
            f"The person {fluff[0]} and {fluff[1]}.",
            "Previously, they " + base[0] + " and " + base[1] + ".",
            "They " + base[2] + ", and " + base[4] + ".",
            "In their free time, the person " + base[3] + " and " + base[5] + "."
        ]
        if T[i] == 1:
            # Treatment is a feature *inside the text*
            parts.insert(2, f"They {treat_phrase}.")
        texts.append(" ".join(parts))

    return texts, T.astype(int), Y.astype(float), U, p

# Quick smoke test
texts, T, Y, U, p_true = simulate_biographies(n=5)
print(texts[0])
print("T:", T[:5], "Y:", Y[:5])


Biography: The person is detail-oriented and likes public speaking. They served in the military for four years. Previously, they worked in finance and budgeting and focused on data and analytics. They earned a BA and supported operations, and has international experience in Europe. In their free time, the person enjoys painting and museums and volunteers with environmental groups.
T: [1 0 0 1 0] Y: [-0.62633984 -3.61121937  5.7760097   4.95511758 -1.71149162]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

@torch.no_grad()
def embed_texts(
    texts,
    model_name="distilbert-base-uncased",
    pooling="cls",   # "cls" | "last" | "mean"
    max_length=256,
    batch_size=32,
    device=None
):
    """
    Returns: embeddings (n, d) as a numpy array.

    pooling:
      - "cls": encoder models with [CLS] token representation
      - "last": decoder-only (or generic) last token representation
      - "mean": mean-pool across tokens (mask-aware)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    tok = AutoTokenizer.from_pretrained(model_name) # from a pre-trained model for tokenization
    if tok.pad_token is None: # for decode only models (e.g., GPT-2, as they don't have a pad token by default)
        tok.pad_token = tok.eos_token

    model = AutoModel.from_pretrained(model_name, output_hidden_states=True) # load pre-trained model for inference
    model.to(device)
    model.eval()

    all_vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tok(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        out = model(**enc) # forward propagation using the pre-trained model and generate the last hidden state
        # out.last_hidden_state: (B, L, H)
        h = out.last_hidden_state
        attn = enc["attention_mask"]  # (B, L) # as enc outputs two tensors: last_hidden_state and attention_mask

        if pooling == "cls": #transfer (B, L, H) to (B, H) (a vector for a specific text/batch)
            # For BERT-like models, token 0 is typically [CLS]
            vec = h[:, 0, :]
        elif pooling == "last":
            # Take the last non-pad token for each row
            lengths = attn.sum(dim=1)  # (B,)
            idx = (lengths - 1).clamp(min=0)
            vec = h[torch.arange(h.size(0)), idx, :]
        elif pooling == "mean":
            # Mask-aware mean pooling
            mask = attn.unsqueeze(-1)  # (B, L, 1)
            summed = (h * mask).sum(dim=1)
            denom = mask.sum(dim=1).clamp(min=1)
            vec = summed / denom
        else:
            raise ValueError("pooling must be one of: cls, last, mean")

        all_vecs.append(vec.detach().cpu())

    emb = torch.cat(all_vecs, dim=0).numpy() # so the output will be a 
    return emb

# Example usage:
# BERT-ish
R = embed_texts(texts, model_name="distilbert-base-uncased", pooling="cls")
# Decoder-only small model (proxy for Llama-style last-token pooling)
# R = embed_texts(texts, model_name="distilgpt2", pooling="last")


C:\Users\pangu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extract the eigens for each texts

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# as R is now a (B,H) matrix, we need to use a function g(R) to get Z (the deconfounder) 
# why we need de-confounding: the positivity assumption in causal inference \pi(T = 1 \mid R)
def make_deconfounder(R, dz=50):
    scaler = StandardScaler(with_mean=True, with_std=True)
    Rz = scaler.fit_transform(R)
    pca = PCA(n_components=dz, random_state=0)
    Z = pca.fit_transform(Rz)
    return Z, scaler, pca


In [5]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, Ridge
import numpy as np

# establish the doubly robust ML model 
# naive estimators: propensity model \pi_A = P(A\mid g(R)); outcome model: \mu_1(R), \mu_0(R) = Y \mid A = a, Z = g(R)
def dml_ate_crossfit(Z, T, Y, n_splits=5, seed=0):
    """
    Cross-fitted EIF ATE estimator.
    Returns: ate, se, psi (influence values)
    """
    Z = np.asarray(Z)
    T = np.asarray(T).astype(int)
    Y = np.asarray(Y).astype(float)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    psi_all = np.zeros_like(Y, dtype=float)

    eps = 1e-3  # clipping for stability

    for train_idx, test_idx in kf.split(Z):
        Ztr, Zte = Z[train_idx], Z[test_idx]
        Ttr, Tte = T[train_idx], T[test_idx]
        Ytr, Yte = Y[train_idx], Y[test_idx]

        # Propensity model
        e_model = LogisticRegression(max_iter=2000)
        e_model.fit(Ztr, Ttr)
        e = e_model.predict_proba(Zte)[:, 1]
        e = np.clip(e, eps, 1 - eps)

        # Outcome models: separate regressions by treatment
        mu1_model = Ridge(alpha=1.0)
        mu0_model = Ridge(alpha=1.0)

        mu1_model.fit(Ztr[Ttr == 1], Ytr[Ttr == 1])
        mu0_model.fit(Ztr[Ttr == 0], Ytr[Ttr == 0])

        mu1 = mu1_model.predict(Zte)
        mu0 = mu0_model.predict(Zte)

        # Efficient influence function
        psi = (mu1 - mu0) + (Tte * (Yte - mu1) / e) - ((1 - Tte) * (Yte - mu0) / (1 - e))
        psi_all[test_idx] = psi

    ate = psi_all.mean()
    se = psi_all.std(ddof=1) / np.sqrt(len(Y))
    return ate, se, psi_all


In [10]:
def run_replication(
    n=1000,
    model_name="distilgpt2",  # try "distilbert-base-uncased" too
    pooling="last",           # "cls" for BERT, "last" for GPT2-like
    dz=50,
    tau_true=1.0,
    seed=7
):
    texts, T, Y, U, p_true = simulate_biographies(n=n, tau=tau_true, seed=seed)

    # 1) Embed texts
    R = embed_texts(texts, model_name=model_name, pooling=pooling, max_length=256, batch_size=32)

    # 2) Deconfounder Z = g(R)
    Z, scaler, pca = make_deconfounder(R, dz=dz)

    # 3) DML ATE
    ate, se, psi = dml_ate_crossfit(Z, T, Y, n_splits=5, seed=seed)

    print(f"True tau = {tau_true:.3f}")
    print(f"DML ATE  = {ate:.3f} (SE {se:.3f})")
    print(f"95% CI   = [{ate - 1.96*se:.3f}, {ate + 1.96*se:.3f}]")
    return {"texts": texts, "T": T, "Y": Y, "R": R, "Z": Z, "ate": ate, "se": se}

# Example:
results  = run_replication(n=10, model_name="distilbert-base-uncased", pooling="cls", dz=3, tau_true=1.0)
results

True tau = 1.000
DML ATE  = 5.954 (SE 1.828)
95% CI   = [2.371, 9.537]


{'texts': ['Biography: The person is detail-oriented and values teamwork. They served in the military for four years. Previously, they worked in finance and budgeting and focused on data and analytics. They earned a BA and supported operations, and has international experience in Europe. In their free time, the person enjoys painting and museums and volunteers with environmental groups.',
  'Biography: The person values teamwork and likes public speaking. They served in the military for four years. Previously, they worked in finance and budgeting and focused on data and analytics. They earned a BA and supported operations, and has international experience in Asia. In their free time, the person enjoys painting and museums and volunteers with youth programs.',
  'Biography: The person is detail-oriented and enjoys reading history. They served in the military for four years. Previously, they worked in finance and budgeting and focused on community outreach. They earned a BA and supported

In [ ]:
# integrations on multi-modal for causal representation learning can be added here


In [ ]:
# balance loss (fine-tuning for the encoding function g(.)) (CORAL method)
# Z = encoder(X) -> (B, d)
# e_hat = propensity_head(Z) -> (B,)
# y0_hat, y1_hat = outcome_heads(Z) -> (B,), (B,)

loss_t = bce(e_hat, T.float())

# factual outcome loss: only supervise observed potential outcome
y_hat = T * y1_hat + (1 - T) * y0_hat
loss_y = mse(y_hat, Y)

# CORAL balance
Z1 = Z[T==1]; Z0 = Z[T==0]
mu1, mu0 = Z1.mean(0), Z0.mean(0)
C1 = (Z1 - mu1).T @ (Z1 - mu1) / (len(Z1)-1 + 1e-6)
C0 = (Z0 - mu0).T @ (Z0 - mu0) / (len(Z0)-1 + 1e-6)
loss_bal = ((mu1-mu0)**2).mean() + ((C1-C0)**2).mean()

loss = loss_y + lam_t*loss_t + lam_bal*loss_bal
loss.backward()
optimizer.step()
